Load the data

Perform 7310

- Satellite raster data (Sentinel 2)
	- 10 spectral bands, one per file (s2_2020_B02.tif, s2_2020_B03.tif, etc...)
	- Each file s2_2020_BXX.tif: contains an array of size (73, 4513, 5183)=(nb. dates, height, width).

In [ ]:
acquisition = 73
def get_7310(pixel):
    x = pixel[0]
    y = pixel[1]
    pixel_7310 = []
    for band in normalized_data_bands:
        for period in acquisition:
            pixel_7310.append(bands[acquisition, x, y])
    return pixel_7310